In [1]:
import nibabel as nib
import numpy as np

image_path = "/home/fetalusr1/Fetal-Head-Segmentation-master/IMG_20250329_13_1.nii"

nii = nib.load(image_path)
data = nii.get_fdata()

print("=== ORIGINAL ===")
print("Shape:", data.shape)
print("Zooms:", nii.header.get_zooms())
print("Affine:\n", nii.affine)

# Visual sanity: aspect ratio
print("XY ratio:", data.shape[0] / data.shape[1])


=== ORIGINAL ===
Shape: (227, 149, 234)
Zooms: (0.3, 0.3, 0.3)
Affine:
 [[ 0.30000001  0.         -0.         -0.        ]
 [ 0.          0.30000001 -0.         -0.        ]
 [ 0.          0.          0.30000001  0.        ]
 [ 0.          0.          0.          1.        ]]
XY ratio: 1.523489932885906


In [2]:
from inference_utils.processing_utils import read_nifti

slice0 = read_nifti(
    image_path,
    is_CT=False,
    slice_idx=0,
    HW_index=(0, 1)
)

print("\n=== read_nifti slice ===")
print("Type:", type(slice0))
print("Shape:", slice0.shape)

if slice0.ndim == 3:
    print("Channels:", slice0.shape[2])

print("XY ratio:", slice0.shape[0] / slice0.shape[1])



=== read_nifti slice ===
Type: <class 'numpy.ndarray'>
Shape: (1024, 1024, 3)
Channels: 3
XY ratio: 1.0


In [3]:
H, W = slice0.shape[:2]
Z = data.shape[2]

fake_volume = np.zeros((H, W, Z), dtype=np.uint8)

for z in range(Z):
    fake_volume[:, :, z] = (slice0[:, :] > 0).astype(np.uint8)

fake_nii = nib.Nifti1Image(fake_volume, nii.affine, nii.header)
nib.save(fake_nii, "fake_identity.nii.gz")


ValueError: could not broadcast input array from shape (1024,1024,3) into shape (1024,1024)